|Patient ID | Tumor type | Compounds screened | Number of Well Fovs |
|-----------|------------|---------------------|---------------------|
| NF0014     |Neurofibroma (subcutaneous) | 15 | 104 |
| Nf0016 | Neurofibroma, diffuse and plexiform | 15 | 125 |
| NF0018 | Neurofibroma, plexiform, non-contiguous cutaneous | 15 | 162 |
| NF0021 | Neurofibroma, plexiform, non-contiguous cutaneous | 15 | 350 |
| NF0030 | Myopericytoma | 15 | 208 |
| NF0040 | Schwannoma, with degeneration | 20 | 420 |
| SARCO219 | MPNST in association with plexiform neurofibroma | 15 | 199 |
| SARCO361 | Sarcoma, treated. | 15 | 350 |